In [4]:
import pandas as pd
import datetime
from db import MySqlHelper
from basic import get_date_shift, get_yesterday, to_datetime, get_today, check_is_UTC0, timing, logging_channels, date_range, datetime_to_str
from jieba_based import Composer_jieba
from keyword_usertag_report import keyword_usertag_report, delete_expired_rows
import jieba.analyse

In [1]:
def main_update_subscriber_usertag(web_id, date, is_UTC0, jump2gcp, expired_day, jieba_base, stopwords, stopwords_usertag):
    ## fetch subscribed browse record
    # data = fetch_browse_record_yesterday_join(web_id, is_df=False, is_UTC0=is_UTC0)
    expired_date = get_date_shift(date_ref=date, days=-expired_day, to_str=True,
                                  is_UTC0=is_UTC0)  ## set to today + 3 (yesterday+4), preserve 4 days
    ####從不同表讀取一些相關資料#######
    data = fetch_browse_record_join(web_id, date=date, is_df=False)
    n_data = len(data)
    if n_data == 0:
        print('no valid data in dione.subscriber_browse_record')
        return pd.DataFrame()
    ## build usertag DataFrame
    j, data_save = 0, {}
    #####將資料整理 並新增usertag,cut,webid
    for i, d in enumerate(data):
        uuid, code, token, cert_web_id, article_id, title, content, keywords = d
        news = title + ' ' + content
        ## pattern for removing https
        news_clean = jieba_base.filter_str(news, pattern="https:\/\/([0-9a-zA-Z.\/]*)")
        ## pattern for removing symbol, -,+~.
        news_clean = jieba_base.filter_symbol(news_clean)
        ##整理keywors,沒有新增 從清理過後的title和content取得
        if (keywords == '') | (keywords == '_'):
            keyword_list = jieba.analyse.extract_tags(news_clean, topK=80) # TF-IDF
            keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)[:8] #取前8關鍵字
            keywords = ','.join(keyword_list)  ## add keywords  
            is_cut = 1
        else:
            keyword_list = [k.strip() for k in keywords.split(',')]
            keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)
            is_cut = 0
        ##新增usertag
        for keyword in keyword_list:
            data_save[j] = {'web_id': web_id, 'uuid': uuid, 'code': code, 'token': token, 'cert_web_id': cert_web_id,
                            'news': news_clean, 'keywords': keywords, 'usertag': keyword, 'article_id': article_id,
                            'expired_date': expired_date, 'is_cut': is_cut}
            j += 1
        if i % 1000 == 0:
            print(f'finish built {i}/{n_data}')
    ## build DataFrame
    df_map = pd.DataFrame.from_dict(data_save, "index")
    ## filter nonsense data
    df_map = df_map[df_map.usertag != '']

    # ## drop unuse columns and drop duplicates, and save to db
    df_map_save = df_map.drop(columns=['news', 'keywords']).drop_duplicates(subset=['web_id','usertag','uuid','article_id'])
    MySqlHelper.ExecuteUpdatebyChunk(df_map_save, db='missioner', table='test_usertag', chunk_size=100000, is_ssh=jump2gcp)
    ## delete expired data
    # delete_expired_rows(web_id, table='usertag', is_UTC0=is_UTC0, jump2gcp=jump2gcp)
    ### prepare keyword_usertag_report
    df_freq_token = keyword_usertag_report(web_id, expired_date, usertag_table='usertag', report_table='usertag_report',
                                           is_UTC0=is_UTC0, jump2gcp=jump2gcp)

    return df_map_save, df_freq_token

In [25]:
def fetch_browse_record_join(web_id, date, is_df=False):
    date_start = to_datetime(date)
    date_end = date_start - datetime.timedelta(days=-1, seconds=1)  ## pixnet, upmedia, ctnews, cmoney,
    query = \
        f"""
            SELECT 
            s.uuid,
            t.code,
            t.registation_id AS token,
            t.cert_web_id,
            s.article_id,
            l.title,
            l.content,
            l.keywords
        FROM
            subscriber_browse_record s
                INNER JOIN
            article_list l ON s.article_id = l.signature                
                AND s.web_id = '{web_id}'                
                AND s.click_time BETWEEN '{date_start}' AND '{date_end}'
                AND l.web_id = '{web_id}'
                INNER JOIN         
            token_index t ON t.uuid = s.uuid
                AND t.invalid = 0
                AND t.web_id = '{web_id}'            
        """
    print(query)
    data = MySqlHelper('dione').ExecuteSelect(query)
    if is_df:
        df = pd.DataFrame(data, columns=['web_id', 'uuid', 'token', 'cert_web_id', 'article_id', 'title', 'content', 'keywords'])
        return df
    else:
        return data

In [26]:
data = fetch_browse_record_join(web_id, date=date, is_df=False)



            SELECT 
            s.uuid,
            t.code,
            t.registation_id AS token,
            t.cert_web_id,
            s.article_id,
            l.title,
            l.content,
            l.keywords
        FROM
            subscriber_browse_record s
                INNER JOIN
            article_list l ON s.article_id = l.signature                
                AND s.web_id = 'cmoney'                
                AND s.click_time BETWEEN '2022-04-07 00:00:00' AND '2022-04-07 23:59:59'
                AND l.web_id = 'cmoney'
                INNER JOIN         
            token_index t ON t.uuid = s.uuid
                AND t.invalid = 0
                AND t.web_id = 'cmoney'            
        


In [32]:
n_data = len(data)
if n_data == 0:
    print('no valid data in dione.subscriber_browse_record')
    ## build usertag DataFrame
j, data_save = 0, {}

In [33]:
data_save

{}

In [35]:
uuid, code, token, cert_web_id, article_id, title, content, keywords =data[0]

In [38]:
news = title + ' ' + content

In [39]:
news

'他犯1失誤，勞退竟少領22萬！申請退休金也要「挑日子」，別讓自己吃虧了 不同時間請領勞退金，金額可能差很大！適用勞退新制的勞工，只要年滿60歲就可以請領勞退金。由於每年投資收益會在隔年3月時才分配，因此若在年中請領新制退休金時，會用最新公告的收益率以「預分配」方式計算'

In [40]:
news_clean = jieba_base.filter_str(news, pattern="https:\/\/([0-9a-zA-Z.\/]*)")
        ## pattern for removing symbol, -,+~.
news_clean = jieba_base.filter_symbol(news_clean)

In [91]:
data

[('2326dcfb-6470-4ec0-acf2-99414cab252e', '1011', 'dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2fZWEUsbIQw1XOVF9lulDYhdL6vvlbwQ-XlX-CB3JSRJJugq8Gq-A6-o6MYQuNDpc9FKpdvWXrAekeEkmcki5XJKss_FAvqCuKktmXqxiO2JxKhSeU9Xe9', 'cmoney1', 'cmoney_notes_250512', '他犯1失誤，勞退竟少領22萬！申請退休金也要「挑日子」，別讓自己吃虧了', '不同時間請領勞退金，金額可能差很大！適用勞退新制的勞工，只要年滿60歲就可以請領勞退金。由於每年投資收益會在隔年3月時才分配，因此若在年中請領新制退休金時，會用最新公告的收益率以「預分配」方式計算', ''),
 ('7b31d0b0-db90-408b-9c06-3f4eb183250f', '1081', 'fmmwOnvpjl8KxN_Q2QJKVI:APA91bGBQfB6PWwgUY33wqCdXMTzwCvoH3DJs28fRj9D_2zrQ0HnB_OXRotQpP1qxxpUGlOTWOpcSMcxu6hMSzpoz8RJkpMdjmk06jz6wsoIB_kwJmv7_YkrXrVNhlr7-2N2MUxLmNcs', 'cmoney1', 'cmoney_notes_161890', '看起來 越傻的人，越不簡單！ 真正有本事的人，往往都有這 3 種表現', '生活中總是有人討論著誰很聰明，誰很有智慧，聰明智慧往往體現著一個人的為人處世，讓別人高興了，別人會說你聰明，有眼力，惹得別人不高興了，就會說你固執，不可理喻，其實你只想做好自己，並不想討好任何人，繼續看下去……(贊助商連結...)長大後很難回到兒時的純真可是這個社會似乎並不像我們想像的那樣，長大以後很難做到像小時候那樣的單純。長大以後就需要智慧處世，不能什麼都一意孤行，', ''),
 ('d72f6f02-76a1-4839-a4d9-4ad3b830fa96', '1011', 'ejfPnZiO-jR3KZ6utpvsln:APA91bHZdReo5a9462tleLfp_kph0E0dnO8HFMw9h8AIH

In [122]:
j, data_save = 0, {}
for i, d in enumerate(data[:10]):
    uuid, code, token, cert_web_id, article_id, title, content, keywords = d
    news = title + ' ' + content
        ## pattern for removing https
    news_clean = jieba_base.filter_str(news, pattern="https:\/\/([0-9a-zA-Z.\/]*)")
        ## pattern for removing symbol, -,+~.
    news_clean = jieba_base.filter_symbol(news_clean)
    if (keywords == '') | (keywords == '_'):
        keyword_list = jieba.analyse.extract_tags(news_clean, topK=80)
        keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)[:8]           
        keywords = ','.join(keyword_list)  ## add keywords
        is_cut = 1
    else:
        keyword_list = [k.strip() for k in keywords.split(',')]
        keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)
        is_cut = 0
    for keyword in keyword_list:
        data_save[j] = {'web_id': web_id, 'uuid': uuid, 'code': code, 'token': token, 'cert_web_id': cert_web_id,
                            'news': news_clean, 'keywords': keywords, 'usertag': keyword, 'article_id': article_id,
                            'expired_date': expired_date, 'is_cut': is_cut}
        j += 1
    if i % 1000 == 0:
        print(f'finish built {i}/{n_data}')

finish built 0/1163


In [120]:
j, data_save = 0, {}
for i, d in enumerate(data[:10]):
    uuid, code, token, cert_web_id, article_id, title, content, keywords = d
    news = title + ' ' + content
        ## pattern for removing https
    news_clean = jieba_base.filter_str(news, pattern="https:\/\/([0-9a-zA-Z.\/]*)")
        ## pattern for removing symbol, -,+~.
    news_clean = jieba_base.filter_symbol(news_clean)
    if (keywords == '') | (keywords == '_'):
        keyword_list = jieba.analyse.extract_tags(news_clean, topK=80)
        keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)[:8]           
        keywords = ','.join(keyword_list)  ## add keywords
        is_cut = 1
    else:
        keyword_list = [k.strip() for k in keywords.split(',')]
        keyword_list = clea

In [124]:
data_save

{0: {'web_id': 'cmoney',
  'uuid': '2326dcfb-6470-4ec0-acf2-99414cab252e',
  'code': '1011',
  'token': 'dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2fZWEUsbIQw1XOVF9lulDYhdL6vvlbwQ-XlX-CB3JSRJJugq8Gq-A6-o6MYQuNDpc9FKpdvWXrAekeEkmcki5XJKss_FAvqCuKktmXqxiO2JxKhSeU9Xe9',
  'cert_web_id': 'cmoney1',
  'news': '他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可能差很大！適用勞退新制的勞工只要年滿60歲就可以請領勞退金。由於每年投資收益會在隔年3月時才分配因此若在年中請領新制退休金時會用最新公告的收益率以「預分配」方式計算',
  'keywords': '領勞退金,退休金,分配,他犯,少領,60歲,投資收益,勞退新制',
  'usertag': '領勞退金',
  'article_id': 'cmoney_notes_250512',
  'expired_date': '2022-04-11',
  'is_cut': 1},
 1: {'web_id': 'cmoney',
  'uuid': '2326dcfb-6470-4ec0-acf2-99414cab252e',
  'code': '1011',
  'token': 'dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2fZWEUsbIQw1XOVF9lulDYhdL6vvlbwQ-XlX-CB3JSRJJugq8Gq-A6-o6MYQuNDpc9FKpdvWXrAekeEkmcki5XJKss_FAvqCuKktmXqxiO2JxKhSeU9Xe9',
  'cert_web_id': 'cmoney1',
  'news': '他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可能差很大！適用勞退新制的勞工只要年滿60歲就可以請領勞退金。由於每年投資收益會在隔年3月時才

In [119]:
keyword_list

['潤泰新', '外資', '殖利率約', '公司法人', '財哥講股', '投資人應', '公司派', '配發現金股利']

In [ ]:
        if (keywords == '') | (keywords == '_'):
            keyword_list = jieba.analyse.extract_tags(news_clean, topK=80)
            keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)[:8]
            keywords = ','.join(keyword_list)  ## add keywords
            is_cut = 1
        else:
            keyword_list = [k.strip() for k in keywords.split(',')]
            keyword_list = clean_keyword_list(keyword_list, stopwords, stopwords_usertag)
            is_cut = 0

In [78]:
keyword_list = jieba.analyse.extract_tags(news_clean, topK=80)

In [84]:
keyword_list

['領勞退金', '退休金', '分配', '他犯', '少領', '60歲', '投資收益', '勞退新制']

In [85]:
keywords = ','.join(keyword_list)

In [86]:
keywords

'領勞退金,退休金,分配,他犯,少領,60歲,投資收益,勞退新制'

In [82]:
def clean_keyword_list(keyword_list, stopwords, stopwords_usertag):
    keyword_list = Composer_jieba().clean_keyword(keyword_list, stopwords)  ## remove stopwords
    keyword_list = Composer_jieba().clean_keyword(keyword_list, stopwords_usertag)  ## remove stopwords, only for usertag
    keyword_list = Composer_jieba().filter_quantifier(keyword_list)  ## remove number+quantifier, ex: 5.1萬
    keyword_list = Composer_jieba().filter_str_list(keyword_list, pattern="[0-9]{2}")  ## remove 2 digit number
    keyword_list = Composer_jieba().filter_str_list(keyword_list, pattern="[0-9.]*")  ## remove floating
    keyword_list = Composer_jieba().filter_str_list(keyword_list, pattern="[a-z]{1,4}|[A-Z]{2}")  ## remove 1-4 lowercase letter and 2 Upper
    keyword_list = [keyword for keyword in keyword_list if keyword != ''] ## remove blank
    return keyword_list

In [83]:
keyword_list= clean_keyword_list(keyword_list, stopwords, stopwords_usertag)[:8]

In [89]:
        for keyword in keyword_list:
            keywords = ','.join(keyword_list)
            data_save[j] = {'web_id': web_id, 'uuid': uuid, 'code': code, 'token': token, 'cert_web_id': cert_web_id,
                            'news': news_clean, 'keywords': keywords, 'usertag': keyword, 'article_id': article_id,
                            'expired_date': expired_date, 'is_cut': is_cut}
            j += 1

In [90]:
data_save

{0: {'web_id': 'cmoney',
  'uuid': '2326dcfb-6470-4ec0-acf2-99414cab252e',
  'code': '1011',
  'token': 'dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2fZWEUsbIQw1XOVF9lulDYhdL6vvlbwQ-XlX-CB3JSRJJugq8Gq-A6-o6MYQuNDpc9FKpdvWXrAekeEkmcki5XJKss_FAvqCuKktmXqxiO2JxKhSeU9Xe9',
  'cert_web_id': 'cmoney1',
  'news': '他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可能差很大！適用勞退新制的勞工只要年滿60歲就可以請領勞退金。由於每年投資收益會在隔年3月時才分配因此若在年中請領新制退休金時會用最新公告的收益率以「預分配」方式計算',
  'keywords': '',
  'usertag': '領勞退金',
  'article_id': 'cmoney_notes_250512',
  'expired_date': '2022-04-11',
  'is_cut': 1},
 1: {'web_id': 'cmoney',
  'uuid': '2326dcfb-6470-4ec0-acf2-99414cab252e',
  'code': '1011',
  'token': 'dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2fZWEUsbIQw1XOVF9lulDYhdL6vvlbwQ-XlX-CB3JSRJJugq8Gq-A6-o6MYQuNDpc9FKpdvWXrAekeEkmcki5XJKss_FAvqCuKktmXqxiO2JxKhSeU9Xe9',
  'cert_web_id': 'cmoney1',
  'news': '他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可能差很大！適用勞退新制的勞工只要年滿60歲就可以請領勞退金。由於每年投資收益會在隔年3月時才分配因此若在年中請領新制退休金時會用最新公告的收益率以「預分配

In [53]:
keyword_list

['領勞退金', '退休金', '分配', '他犯', '少領', '60歲', '投資收益', '勞退新制']

In [55]:
is_cut=1

In [59]:
keyword_list

['領勞退金', '退休金', '分配', '他犯', '少領', '60歲', '投資收益', '勞退新制']

In [87]:
for keyword in keyword_list:
    data_save[j] = {'web_id': web_id, 'uuid': uuid, 'code': code, 'token': token, 'cert_web_id': cert_web_id,
                            'news': news_clean, 'keywords': keywords, 'usertag': keyword, 'article_id': article_id,
                            'expired_date': expired_date, 'is_cut': is_cut}
    j += 1

In [88]:
data_save

{0: {'web_id': 'cmoney',
  'uuid': '2326dcfb-6470-4ec0-acf2-99414cab252e',
  'code': '1011',
  'token': 'dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2fZWEUsbIQw1XOVF9lulDYhdL6vvlbwQ-XlX-CB3JSRJJugq8Gq-A6-o6MYQuNDpc9FKpdvWXrAekeEkmcki5XJKss_FAvqCuKktmXqxiO2JxKhSeU9Xe9',
  'cert_web_id': 'cmoney1',
  'news': '他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可能差很大！適用勞退新制的勞工只要年滿60歲就可以請領勞退金。由於每年投資收益會在隔年3月時才分配因此若在年中請領新制退休金時會用最新公告的收益率以「預分配」方式計算',
  'keywords': '',
  'usertag': '領勞退金',
  'article_id': 'cmoney_notes_250512',
  'expired_date': '2022-04-11',
  'is_cut': 1},
 1: {'web_id': 'cmoney',
  'uuid': '2326dcfb-6470-4ec0-acf2-99414cab252e',
  'code': '1011',
  'token': 'dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2fZWEUsbIQw1XOVF9lulDYhdL6vvlbwQ-XlX-CB3JSRJJugq8Gq-A6-o6MYQuNDpc9FKpdvWXrAekeEkmcki5XJKss_FAvqCuKktmXqxiO2JxKhSeU9Xe9',
  'cert_web_id': 'cmoney1',
  'news': '他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可能差很大！適用勞退新制的勞工只要年滿60歲就可以請領勞退金。由於每年投資收益會在隔年3月時才分配因此若在年中請領新制退休金時會用最新公告的收益率以「預分配

In [62]:
 df_map = pd.DataFrame.from_dict(data_save, "index")

In [63]:
df_map

web_id                                  uuid  code  \
0  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
1  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
2  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
3  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
4  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
5  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
6  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
7  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   

                                               token cert_web_id  \
0  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
1  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
2  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
3  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
4  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
5  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
6  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
7  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   

                                                news keywords usertag  \
0  他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...             領勞退金   
1  他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...              退休金   
2  他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...               分配   
3  他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...               他犯   
4  他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...               少領   
5  他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...              60歲   
6  他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...             投資收益   
7  他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...             勞退新制   

            article_id expired_date  is_cut  
0  cmoney_notes_250512   2022-04-11       1  
1  cmoney_notes_250512   2022-04-11       1  
2  cmoney_notes_250512   2022-04-11       1  
3  cmoney_notes_250512   2022-04-11       1  
4  cmoney_notes_250512   2022-04-11       1  
5  cmoney_notes_250512   2022-04-11       1  
6  cmoney_notes_250512   2022-04-11       1  
7  cmoney_notes_250512   2022-04-11       1

In [ ]:
    df_map = df_map[df_map.usertag != '']

    # ## drop unuse columns and drop duplicates, and save to db
    df_map_save = df_map.drop(columns=['news', 'keywords']).drop_duplicates(subset=['web_id','usertag','uuid','article_id'])
    MySqlHelper.ExecuteUpdatebyChunk(df_map_save, db='missioner', table='test_usertag', chunk_size=100000, is_ssh=jump2gcp)
    ## delete expired data
    # delete_expired_rows(web_id, table='usertag', is_UTC0=is_UTC0, jump2gcp=jump2gcp)
    ### prepare keyword_usertag_report
    df_freq_token = keyword_usertag_report(web_id, expired_date, usertag_table='usertag', report_table='usertag_report',
                                           is_UTC0=is_UTC0, jump2gcp=jump2gcp)

    return df_map_save, df_freq_token   

In [134]:
df_map = df_map[df_map.usertag != '']

In [135]:
df_map

web_id                                  uuid  code  \
0   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
1   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
2   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
3   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
4   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
..     ...                                   ...   ...   
75  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   
76  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   
77  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   
78  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   
79  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   

                                                token cert_web_id  \
0   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
1   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
2   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
3   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
4   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1   
..                                                ...         ...   
75  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1   
76  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1   
77  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1   
78  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1   
79  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1   

                                                 news  \
0   他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...   
1   他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...   
2   他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...   
3   他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...   
4   他犯1失誤勞退竟少領22萬！申請退休金也要「挑日子」別讓自己吃虧了 不同時間請領勞退金金額可...   
..                                                ...   
75  【財哥講股】潤泰新(9945)：殖利率超過8％！現在還適合存股嗎？ 🥇🥇🥇※警語：全文發佈於...   
76  【財哥講股】潤泰新(9945)：殖利率超過8％！現在還適合存股嗎？ 🥇🥇🥇※警語：全文發佈於...   
77  【財哥講股】潤泰新(9945)：殖利率超過8％！現在還適合存股嗎？ 🥇🥇🥇※警語：全文發佈於...   
78  【財哥講股】潤泰新(9945)：殖利率超過8％！現在還適合存股嗎？ 🥇🥇🥇※警語：全文發佈於...   
79  【財哥講股】潤泰新(9945)：殖利率超過8％！現在還適合存股嗎？ 🥇🥇🥇※警語：全文發佈於...   

                                 keywords usertag           article_id  \
0         領勞退金,退休金,分配,他犯,少領,60歲,投資收益,勞退新制    領勞退金  cmoney_notes_250512   
1         領勞退金,退休金,分配,他犯,少領,60歲,投資收益,勞退新制     退休金  cmoney_notes_250512   
2         領勞退金,退休金,分配,他犯,少領,60歲,投資收益,勞退新制      分配  cmoney_notes_250512   
3         領勞退金,退休金,分配,他犯,少領,60歲,投資收益,勞退新制      他犯  cmoney_notes_250512   
4         領勞退金,退休金,分配,他犯,少領,60歲,投資收益,勞退新制      少領  cmoney_notes_250512   
..                                    ...     ...                  ...   
75  潤泰新,外資,殖利率約,公司法人,財哥講股,投資人應,公司派,配發現金股利    公司法人  cmoney_notes_393274   
76  潤泰新,外資,殖利率約,公司法人,財哥講股,投資人應,公司派,配發現金股利    財哥講股  cmoney_notes_393274   
77  潤泰新,外資,殖利率約,公司法人,財哥講股,投資人應,公司派,配發現金股利    投資人應  cmoney_notes_393274   
78  潤泰新,外資,殖利率約,公司法人,財哥講股,投資人應,公司派,配發現金股利     公司派  cmoney_notes_393274   
79  潤泰新,外資,殖利率約,公司法人,財哥講股,投資人應,公司派,配發現金股利  配發現金股利  cmoney_notes_393274   

   expired_date  is_cut  
0    2022-04-11       1  
1    2022-04-11       1  
2    2022-04-11       1  
3    2022-04-11       1  
4    2022-04-11       1  
..          ...     ...  
75   2022-04-11       1  
76   2022-04-11       1  
77   2022-04-11       1  
78   2022-04-11       1  
79   2022-04-11       1  

[80 rows x 11 columns]

In [136]:
df_map_save = df_map.drop(columns=['news', 'keywords']).drop_duplicates(subset=['web_id','usertag','uuid','article_id'])

In [70]:
df_map.drop(columns=['news', 'keywords']).drop_duplicates(subset=['web_id'])

web_id                                  uuid  code  \
0  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   

                                               token cert_web_id usertag  \
0  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1    領勞退金   

            article_id expired_date  is_cut  
0  cmoney_notes_250512   2022-04-11       1

In [137]:
df_map_save

web_id                                  uuid  code  \
0   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
1   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
2   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
3   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
4   cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
..     ...                                   ...   ...   
75  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   
76  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   
77  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   
78  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   
79  cmoney  1bd1cebe-87f0-4d72-9f9d-82683e6330b4  1011   

                                                token cert_web_id usertag  \
0   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1    領勞退金   
1   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1     退休金   
2   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1      分配   
3   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1      他犯   
4   dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1      少領   
..                                                ...         ...     ...   
75  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1    公司法人   
76  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1    財哥講股   
77  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1    投資人應   
78  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1     公司派   
79  cKc-Hcorg8f2CkDQWU5N7S:APA91bHxKrA28H5j74hgXT8...     cmoney1  配發現金股利   

             article_id expired_date  is_cut  
0   cmoney_notes_250512   2022-04-11       1  
1   cmoney_notes_250512   2022-04-11       1  
2   cmoney_notes_250512   2022-04-11       1  
3   cmoney_notes_250512   2022-04-11       1  
4   cmoney_notes_250512   2022-04-11       1  
..                  ...          ...     ...  
75  cmoney_notes_393274   2022-04-11       1  
76  cmoney_notes_393274   2022-04-11       1  
77  cmoney_notes_393274   2022-04-11       1  
78  cmoney_notes_393274   2022-04-11       1  
79  cmoney_notes_393274   2022-04-11       1  

[80 rows x 9 columns]

In [68]:
subset=['web_id','usertag','uuid','article_id']

In [132]:
df_map_save

web_id                                  uuid  code  \
0  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
1  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
2  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
3  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
4  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
5  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
6  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   
7  cmoney  2326dcfb-6470-4ec0-acf2-99414cab252e  1011   

                                               token cert_web_id usertag  \
0  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1    領勞退金   
1  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1     退休金   
2  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1      分配   
3  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1      他犯   
4  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1      少領   
5  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1     60歲   
6  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1    投資收益   
7  dMn8I03RPunhP7XWpMAV09:APA91bFFv7LuhOQc0eYKFw2...     cmoney1    勞退新制   

            article_id expired_date  is_cut  
0  cmoney_notes_250512   2022-04-11       1  
1  cmoney_notes_250512   2022-04-11       1  
2  cmoney_notes_250512   2022-04-11       1  
3  cmoney_notes_250512   2022-04-11       1  
4  cmoney_notes_250512   2022-04-11       1  
5  cmoney_notes_250512   2022-04-11       1  
6  cmoney_notes_250512   2022-04-11       1  
7  cmoney_notes_250512   2022-04-11       1